In [2]:
import pandas as pd
import sqlalchemy as sa

In [3]:
# Credentials SQL
host = 'comunidade-ds-postgres.c50pcakiuwi3.us-east-1.rds.amazonaws.com'
port = 5432
database = 'comunidadedsdb'
username = 'member'
pwd = 'cdspa'

In [4]:
query = """
        SELECT *
        FROM pa004.users
"""

In [7]:
string_connect = 'postgresql://' + username + ':' + pwd + '@' + host + ':' + str(port) + '/' + database
engine = sa.create_engine(string_connect)
conn1 = engine.connect()
df = pd.read_sql(query, conn1)
df.head()

,id,gender,age,region_code,policy_sales_channel
0,1,Male,44,28.0,26.0
1,2,Male,76,3.0,26.0
2,3,Male,47,28.0,26.0
3,4,Male,21,11.0,152.0
4,5,Female,29,41.0,152.0


In [8]:
df

,id,gender,age,region_code,policy_sales_channel
0,1,Male,44,28.0,26.0
1,2,Male,76,3.0,26.0
2,3,Male,47,28.0,26.0
3,4,Male,21,11.0,152.0
4,5,Female,29,41.0,152.0
...,...,...,...,...,...
381104,381105,Male,74,26.0,26.0
381105,381106,Male,30,37.0,152.0
381106,381107,Male,21,30.0,160.0
381107,381108,Female,68,14.0,124.0
